In [3]:
!pip install torch torchvision torchaudio
!pip install git+https://github.com/openai/CLIP.git
!pip install matplotlib



  Cloning https://github.com/openai/CLIP.git to c:\users\shazi\appdata\local\temp\pip-req-build-dawoivp5
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369633 sha256=c4e93c0f6cc681181739076532a62e7ef3f1a466b26d25dd0965eb6973330cf3
  Stored in directory: C:\Users\shazi\AppData\Local\Temp\pip-ephem-wheel-cache-h1l96kxl\wheels\35\3e\df\3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\shazi\AppData\Local\Temp\pip-req-build-dawoivp5'


In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import clip  # OpenAI's CLIP
from tqdm import tqdm

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load CLIP model and preprocessing
clip_model, _ = clip.load("ViT-B/32", device=device)

# Custom transform to 128x128 + CLIP normalization
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# Replace this with your dataset path
data_dir = "C:/Users/shazi/OneDrive/Desktop/VS Code/fyp/skintypepatches 128x128"  # example: "dataset/"

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Print class names
print("Classes:", dataset.classes)

# Freeze CLIP vision encoder layers
for param in clip_model.visual.parameters():
    param.requires_grad = False

# Build the model using CLIP ViT encoder
class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits

# Initialize model
model = CLIPSkinClassifier(clip_model, num_classes=3).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Metrics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            loop.set_postfix(loss=loss.item(), acc=100. * correct / total)

        print(f"Epoch {epoch+1}: Loss={running_loss/len(dataloader):.4f}, Accuracy={100. * correct/total:.2f}%")

# Train
train_model(model, train_loader, criterion, optimizer, epochs=10)


Using device: cuda


100%|████████████████████████████████████████| 338M/338M [06:24<00:00, 920kiB/s]


Classes: ['dryPatches', 'normalPatch', 'oilyPatch']


Epoch 1/10:   0%|          | 0/1115 [00:00<?, ?it/s]


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.cuda.HalfTensor) should be the same

In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import clip  # OpenAI's CLIP
from tqdm import tqdm

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load CLIP model and preprocessing
clip_model, _ = clip.load("ViT-B/32", device=device)

# Custom transform to 128x128 + CLIP normalization
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# Your dataset path
data_dir = "C:/Users/shazi/OneDrive/Desktop/VS Code/fyp/skintypepatches 128x128"

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Create DataLoader
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Print class names
print("Classes:", dataset.classes)

# Freeze CLIP vision encoder layers
for param in clip_model.visual.parameters():
    param.requires_grad = False

# Build model with CLIP ViT encoder
class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = x.half()  # Ensure input is float16
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits

# Initialize model and convert to half precision
model = CLIPSkinClassifier(clip_model, num_classes=3).to(device).half()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        for images, labels in loop:
            images = images.to(device).half()  # Match input type to model (float16)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Metrics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            loop.set_postfix(loss=loss.item(), acc=100. * correct / total)

        print(f"Epoch {epoch+1}: Loss={running_loss/len(dataloader):.4f}, Accuracy={100. * correct/total:.2f}%")

# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=10)


Using device: cuda
Classes: ['dryPatches', 'normalPatch', 'oilyPatch']


Epoch 1/10:   0%|          | 0/1115 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (17) must match the size of tensor b (50) at non-singleton dimension 1

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import clip  # OpenAI's CLIP
from tqdm import tqdm

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load CLIP model and preprocessing
clip_model, _ = clip.load("ViT-B/32", device=device)

# Custom transform to 224x224 + CLIP normalization (corrected size)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 for CLIP compatibility
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# Replace this with your dataset path
data_dir = r"C:\Users\shazi\OneDrive\Desktop\VS Code\fyp\skintypepatches 128x128"  # example: "dataset/"

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Print class names
print("Classes:", dataset.classes)

# Freeze CLIP vision encoder layers
for param in clip_model.visual.parameters():
    param.requires_grad = False

# Build the model using CLIP ViT encoder
class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits

# Initialize model
model = CLIPSkinClassifier(clip_model, num_classes=3).to(device).float()  # Ensure model is in float32

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        for images, labels in tqdm(loop):
            images, labels = images.to(device).float(), labels.to(device)  # Ensure input is in float32

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Metrics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            loop.set_postfix(loss=loss.item(), acc=100. * correct / total)

        print(f"Epoch {epoch+1}: Loss={running_loss/len(dataloader):.4f}, Accuracy={100. * correct/total:.2f}%")

# Train
train_model(model, train_loader, criterion, optimizer, epochs=10)


Using device: cuda
Classes: ['dryPatches', 'normalPatch', 'oilyPatch']


100%|██████████| 1115/1115 [06:50<00:00,  2.72it/s]


Epoch 1: Loss=1.0672, Accuracy=42.35%


  5%|▍         | 54/1115 [00:18<06:03,  2.92it/s]


KeyboardInterrupt: 